# Step 1: Downloaded 2 files from TUEG, try to copy the Braincode TUH class, step by step and function by function

In [1]:
import mne 
path = '/rds/general/user/nm2318/home/projects/scott_data_tuh/live/tuh_eeg/v2.0.0/edf/000/aaaaaaaa/s001_2015_12_30/01_tcp_ar/aaaaaaaa_s001_t000.edf'
raw_tueeg_test = mne.io.read_raw_edf(path, preload=True)

Extracting EDF parameters from /rds/general/user/nm2318/home/projects/scott_data_tuh/live/tuh_eeg/v2.0.0/edf/000/aaaaaaaa/s001_2015_12_30/01_tcp_ar/aaaaaaaa_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 323839  =      0.000 ...  1264.996 secs...


In [2]:
# Examine the data and what it contains
key_non_null = [key for key in raw_tueeg_test.info.keys() if ((raw_tueeg_test.info[key] is not None) and (raw_tueeg_test.info[key] != []))]
for key in key_non_null:
    print(key,'  ',raw_tueeg_test.info[key])

highpass    0.0
lowpass    128.0
meas_date    2015-12-30 14:44:21+00:00
subject_info    {'his_id': 'aaaaaaaa', 'sex': 1, 'last_name': 'aaaaaaaa'}
chs    [{'cal': 1.0, 'logno': 1, 'scanno': 1, 'range': 1.0, 'unit_mul': 0 (FIFF_UNITM_NONE), 'ch_name': 'EEG FP1-REF', 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FIFFV_COORD_HEAD), 'coil_type': 1 (FIFFV_COIL_EEG), 'kind': 2 (FIFFV_EEG_CH), 'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])}, {'cal': 1.0, 'logno': 2, 'scanno': 2, 'range': 1.0, 'unit_mul': 0 (FIFF_UNITM_NONE), 'ch_name': 'EEG FP2-REF', 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FIFFV_COORD_HEAD), 'coil_type': 1 (FIFFV_COIL_EEG), 'kind': 2 (FIFFV_EEG_CH), 'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])}, {'cal': 1.0, 'logno': 3, 'scanno': 3, 'range': 1.0, 'unit_mul': 0 (FIFF_UNITM_NONE), 'ch_name': 'EEG F3-REF', 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FIFFV_COORD_HEAD), 'coil_type': 1 (FIFFV_COIL_EEG), 'kind': 2 (FIFFV_

## _read_edf_header and _parse_age_and_gender_from_edf_header

In [3]:
f = open(path, "rb")
header = f.read(88)
f.close()

In [4]:
import re
patient_id = header[8:].decode("ascii")

age = -1
found_age = re.findall(r"Age:(\d+)", patient_id)
if len(found_age) == 1:
    age = int(found_age[0])
    
gender = "X"
found_gender = re.findall(r"\s([F|M])\s", patient_id)
if len(found_gender) == 1:
    gender = found_gender[0]


In [5]:
age

37

In [6]:
gender

'M'

In [7]:
patient_id

'aaaaaaaa M XX-XXX-XXXX aaaaaaaa Age:37                                          '

## _read_physician_report
This does not work bevause it looks for txt files in the directory, where there is only edf files. In version. 2 of TUH corpus, the reports have been removed.

In [8]:
import os
import glob
import warnings

directory = os.path.dirname(path)
txt_file = glob.glob(os.path.join(directory, '**/*.txt'), recursive=True)
# check that there is at most one txt file in the same directory
assert len(txt_file) in [0, 1]
report = ''
if txt_file:
    txt_file = txt_file[0]
    # somewhere in the corpus, encoding apparently changed
    # first try to read as utf-8, if it does not work use latin-1
    try:
        with open(txt_file, 'r', encoding='utf-8') as f:
            report = f.read()
    except UnicodeDecodeError:
        with open(txt_file, 'r', encoding='latin-1') as f:
            report = f.read()
            
if not report:
    raise RuntimeError(f'Could not read physician report ({txt_file}). '
                           f'Disable option or choose appropriate directory.')

RuntimeError: Could not read physician report ([]). Disable option or choose appropriate directory.

In [9]:
#No txt files found
txt_file

[]

## _parse_description_from_file_path

WORKS WITH VERSION (abnormal and v_number >= 3) or ((not abnormal) and v_number >= 2)

New file path structure for versions after december 2022, expect file paths as
tuh_eeg/v2.0.0/edf/000/aaaaaaaa/s001_2015_12_30/01_tcp_ar/aaaaaaaa_s001_t000.edf

or for abnormal:
tuh_eeg_abnormal/v3.0.0/edf/train/normal/01_tcp_ar/aaaaaaav_s004_t000.edf

In [10]:
file_path = os.path.normpath(path)
tokens = file_path.split(os.sep)
tokens

['',
 'rds',
 'general',
 'user',
 'nm2318',
 'home',
 'projects',
 'scott_data_tuh',
 'live',
 'tuh_eeg',
 'v2.0.0',
 'edf',
 '000',
 'aaaaaaaa',
 's001_2015_12_30',
 '01_tcp_ar',
 'aaaaaaaa_s001_t000.edf']

In [11]:
#play around with token, look at them
tokens[-3].split('_')[0][0]

's'

## _read_date

Here, do you want a new file to be create if it does not exist? maybe useless, also, maybe remove the lines looking for a file as they dont exist




In [12]:
import pandas as pd

file_path = path
date_path = file_path.replace('.edf', '_date.txt')
 # if date file exists, read it
if os.path.exists(date_path):
    description = pd.read_json(date_path, typ='series').to_dict()
# otherwise read edf file, extract date and store to file
else:
    raw = mne.io.read_raw_edf(file_path, preload=False, verbose='error')
    description = {
        'year': raw.info['meas_date'].year,
        'month': raw.info['meas_date'].month,
        'day': raw.info['meas_date'].day,
    }
    # if the txt file storing the recording date does not exist, create it
    try:
        pd.Series(description).to_json(date_path)
    except OSError:
        warnings.warn(f'Cannot save date file to {date_path}. 'f'This might slow down creation of the dataset.')